In [1]:
# Common imports
import numpy as np
import numpy.random as rnd
import os
import pandas as pd

# to make this notebook's output stable across runs
rnd.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
csv_path = 'MIMIC II/ADMISSIONS.csv'
admissions = pd.read_csv(csv_path)

csv_path = 'MIMIC II/PATIENTS.csv'
patients = pd.read_csv(csv_path)

csv_path = 'MIMIC II/ICUSTAYS.csv'
icustays = pd.read_csv(csv_path)

csv_path = 'MIMIC II/CALLOUT.csv'
callout = pd.read_csv(csv_path)

admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [3]:
#Time to combine some data frames
#start by cleaning up the data frames...
admissions = admissions.drop(['ROW_ID', 'DEATHTIME', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'EDREGTIME', 'EDOUTTIME', 'EDOUTTIME', 'HAS_CHARTEVENTS_DATA'], axis=1)
callout = callout.drop(['ROW_ID', 'SUBMIT_WARDID', 'SUBMIT_CAREUNIT', 'CURR_WARDID', 'SUBMIT_CAREUNIT', 'CURR_WARDID', 'CALLOUT_WARDID', 'CALLOUT_STATUS', 'DISCHARGE_WARDID', 'ACKNOWLEDGE_STATUS', 'CREATETIME', 'UPDATETIME', 'ACKNOWLEDGETIME', 'OUTCOMETIME', 'FIRSTRESERVATIONTIME', 'CURRENTRESERVATIONTIME'], axis = 1)
icustays = icustays.drop(['ROW_ID', 'FIRST_WARDID', 'LAST_WARDID'], axis = 1)
patients = patients.drop(['ROW_ID', 'DOD', 'DOD_HOSP', 'DOD_SSN'], axis = 1)



In [4]:
#x = admissions.merge(icustays,on='HADM_ID').merge(callout,on='HADM_ID').merge(patients, on='SUBJECT_ID')
#x.info()

In [5]:
import functools
dfs = [admissions, callout, icustays]

In [6]:
df_final =  functools.reduce(lambda left,right: pd.merge(left,right,on='HADM_ID'), dfs)
    #replace repeat subject_IDs
df_final = df_final.drop(['SUBJECT_ID_y', 'SUBJECT_ID'], axis = 1)
df_final.columns = df_final.columns.str.replace('SUBJECT_ID_x','SUBJECT_ID')

In [7]:
pd.set_option('display.max_columns', None)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40482 entries, 0 to 40481
Data columns (total 26 columns):
SUBJECT_ID              40482 non-null int64
HADM_ID                 40482 non-null int64
ADMITTIME               40482 non-null object
DISCHTIME               40482 non-null object
ADMISSION_TYPE          40482 non-null object
ADMISSION_LOCATION      40482 non-null object
DISCHARGE_LOCATION      40482 non-null object
INSURANCE               40482 non-null object
ETHNICITY               40482 non-null object
DIAGNOSIS               40481 non-null object
HOSPITAL_EXPIRE_FLAG    40482 non-null int64
CURR_CAREUNIT           40481 non-null object
CALLOUT_SERVICE         40482 non-null object
REQUEST_TELE            40482 non-null int64
REQUEST_RESP            40482 non-null int64
REQUEST_CDIFF           40482 non-null int64
REQUEST_MRSA            40482 non-null int64
REQUEST_VRE             40482 non-null int64
CALLOUT_OUTCOME         40482 non-null object
ICUSTAY_ID              4

In [8]:
dfs_f = [df_final, patients]
df_final2 = functools.reduce(lambda left, right: pd.merge(left,right,on='SUBJECT_ID'), dfs_f)
df_final2.columns = df_final2.columns.str.replace('_x','')
df_final2.columns = df_final2.columns.str.replace('_y','')
#We dont need subject_id anymore
#df_final2 = df_final2.drop('SUBJECT_ID', axis = 1)
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40482 entries, 0 to 40481
Data columns (total 29 columns):
SUBJECT_ID              40482 non-null int64
HADM_ID                 40482 non-null int64
ADMITTIME               40482 non-null object
DISCHTIME               40482 non-null object
ADMISSION_TYPE          40482 non-null object
ADMISSION_LOCATION      40482 non-null object
DISCHARGE_LOCATION      40482 non-null object
INSURANCE               40482 non-null object
ETHNICITY               40482 non-null object
DIAGNOSIS               40481 non-null object
HOSPITAL_EXPIRE_FLAG    40482 non-null int64
CURR_CAREUNIT           40481 non-null object
CALLOUT_SERVICE         40482 non-null object
REQUEST_TELE            40482 non-null int64
REQUEST_RESP            40482 non-null int64
REQUEST_CDIFF           40482 non-null int64
REQUEST_MRSA            40482 non-null int64
REQUEST_VRE             40482 non-null int64
CALLOUT_OUTCOME         40482 non-null object
ICUSTAY_ID              4

In [9]:
#Split into Training (80%) and Test (20%) sets
#from sklearn.model_selection import train_test_split

#train_set, test_set = train_test_split(df_final2, test_size=0.2, random_state=42)
#test_set.head()
df_final2.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,CURR_CAREUNIT,CALLOUT_SERVICE,REQUEST_TELE,REQUEST_RESP,REQUEST_CDIFF,REQUEST_MRSA,REQUEST_VRE,CALLOUT_OUTCOME,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,GENDER,DOB,EXPIRE_FLAG
0,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,WHITE,BRAIN MASS,0,SICU,NSURG,1,0,0,0,0,Discharged,234044,metavision,SICU,SICU,2157-10-21 11:40:38,2157-10-22 16:08:48,1.1862,M,2082-07-17 00:00:00,0
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,WHITE,CHEST PAIN\CATH,0,MICU,CCU,0,0,0,0,0,Discharged,263086,carevue,MICU,MICU,2186-07-18 18:10:49,2186-07-19 11:27:20,0.7198,M,1886-07-18 00:00:00,1
2,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,WHITE,BRADYCARDIA,0,CCU,CCU,1,0,0,0,0,Discharged,290505,metavision,CCU,CCU,2191-02-23 05:25:32,2191-02-24 19:24:10,1.5824,M,1886-07-18 00:00:00,1
3,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,CSRU,CSURG,1,0,0,0,0,Discharged,280987,carevue,CSRU,CSRU,2131-05-04 10:29:14,2131-05-05 13:07:03,1.1096,M,2061-08-17 00:00:00,0
4,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,WHITE,CHEST PAIN/SHORTNESS OF BREATH,0,CSRU,CSURG,1,0,0,0,0,Discharged,211200,carevue,CSRU,CSRU,2131-05-16 23:18:26,2131-05-23 19:56:11,6.8595,M,2061-08-17 00:00:00,0


In [10]:
#lets break this apart a little bit
#FirstDF: times, patient info, diagnosis, mortality flag, LOS
time_data = df_final2[[ 'ADMITTIME', 'DISCHTIME', 'LOS']].copy()
time_data.head()

,ADMITTIME,DISCHTIME,LOS
0,2157-10-18 19:34:00,2157-10-25 14:00:00,1.1862
1,2186-07-18 16:46:00,2186-07-20 16:00:00,0.7198
2,2191-02-23 05:23:00,2191-02-25 20:20:00,1.5824
3,2131-04-30 07:15:00,2131-05-08 14:00:00,1.1096
4,2131-05-12 19:49:00,2131-05-25 13:30:00,6.8595


In [11]:
#SecondDF: categorical data
cat_data = df_final2.drop(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME', 'DOB', 'LOS', 'EXPIRE_FLAG'], axis = 1)
cat_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40482 entries, 0 to 40481
Data columns (total 19 columns):
ADMISSION_TYPE          40482 non-null object
ADMISSION_LOCATION      40482 non-null object
DISCHARGE_LOCATION      40482 non-null object
INSURANCE               40482 non-null object
ETHNICITY               40482 non-null object
DIAGNOSIS               40481 non-null object
HOSPITAL_EXPIRE_FLAG    40482 non-null int64
CURR_CAREUNIT           40481 non-null object
CALLOUT_SERVICE         40482 non-null object
REQUEST_TELE            40482 non-null int64
REQUEST_RESP            40482 non-null int64
REQUEST_CDIFF           40482 non-null int64
REQUEST_MRSA            40482 non-null int64
REQUEST_VRE             40482 non-null int64
CALLOUT_OUTCOME         40482 non-null object
DBSOURCE                40482 non-null object
FIRST_CAREUNIT          40482 non-null object
LAST_CAREUNIT           40482 non-null object
GENDER                  40482 non-null object
dtypes: int64(6), objec

In [12]:
#lets get the dates into weekdays
import datetime

admTime = time_data['ADMITTIME']
dischTime = time_data['DISCHTIME']

def get_day_of_week (value):
    date = []
    int_date = []
    day_of_week = []
    
    #replace dashes and colons with spaces to make breaking the string up easier
    value = value.replace('-', ' ')
    value = value.replace(':', ' ')
    
    #split into a string outputting [year, month, day, hour, minutes, seconds]
    date = value.split(' ')
    
    #delete hour/min/sec, convert to integers
    del date[3:6]
    int_date = [int(i) for i in date]
    
    #get day of week and return it
    day_of_week = datetime.date(int_date[0], int_date[1], int_date[2]).weekday()
    
    return day_of_week


admTime_weekday = [get_day_of_week(value) for value in admTime]
dischTime_weekday = [get_day_of_week(value) for value in dischTime]

weekday_data = pd.DataFrame({'admTime_weekday' : admTime_weekday, 'dischTime_weekday' : dischTime_weekday})

weekday_data.head()

,admTime_weekday,dischTime_weekday
0,1,1
1,1,3
2,2,4
3,0,1
4,5,4


In [13]:
#Organize dataframes back into categorical and numerical dataframes
cat_data = pd.concat([cat_data, weekday_data], axis=1)
cat_data.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,CURR_CAREUNIT,CALLOUT_SERVICE,REQUEST_TELE,REQUEST_RESP,REQUEST_CDIFF,REQUEST_MRSA,REQUEST_VRE,CALLOUT_OUTCOME,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,GENDER,admTime_weekday,dischTime_weekday
0,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,WHITE,BRAIN MASS,0,SICU,NSURG,1,0,0,0,0,Discharged,metavision,SICU,SICU,M,1,1
1,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,WHITE,CHEST PAIN\CATH,0,MICU,CCU,0,0,0,0,0,Discharged,carevue,MICU,MICU,M,1,3
2,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,WHITE,BRADYCARDIA,0,CCU,CCU,1,0,0,0,0,Discharged,metavision,CCU,CCU,M,2,4
3,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,CSRU,CSURG,1,0,0,0,0,Discharged,carevue,CSRU,CSRU,M,0,1
4,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,WHITE,CHEST PAIN/SHORTNESS OF BREATH,0,CSRU,CSURG,1,0,0,0,0,Discharged,carevue,CSRU,CSRU,M,5,4


In [14]:
num_data = time_data.drop(['ADMITTIME', 'DISCHTIME'], axis = 1)
num_data.head()

,LOS
0,1.1862
1,0.7198
2,1.5824
3,1.1096
4,6.8595


In [15]:
cat_data = cat_data.applymap(str)
cat_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40482 entries, 0 to 40481
Data columns (total 21 columns):
ADMISSION_TYPE          40482 non-null object
ADMISSION_LOCATION      40482 non-null object
DISCHARGE_LOCATION      40482 non-null object
INSURANCE               40482 non-null object
ETHNICITY               40482 non-null object
DIAGNOSIS               40482 non-null object
HOSPITAL_EXPIRE_FLAG    40482 non-null object
CURR_CAREUNIT           40482 non-null object
CALLOUT_SERVICE         40482 non-null object
REQUEST_TELE            40482 non-null object
REQUEST_RESP            40482 non-null object
REQUEST_CDIFF           40482 non-null object
REQUEST_MRSA            40482 non-null object
REQUEST_VRE             40482 non-null object
CALLOUT_OUTCOME         40482 non-null object
DBSOURCE                40482 non-null object
FIRST_CAREUNIT          40482 non-null object
LAST_CAREUNIT           40482 non-null object
GENDER                  40482 non-null object
admTime_weekday  

In [16]:
all_data = pd.concat([cat_data, num_data], axis = 1)

In [17]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40482 entries, 0 to 40481
Data columns (total 22 columns):
ADMISSION_TYPE          40482 non-null object
ADMISSION_LOCATION      40482 non-null object
DISCHARGE_LOCATION      40482 non-null object
INSURANCE               40482 non-null object
ETHNICITY               40482 non-null object
DIAGNOSIS               40482 non-null object
HOSPITAL_EXPIRE_FLAG    40482 non-null object
CURR_CAREUNIT           40482 non-null object
CALLOUT_SERVICE         40482 non-null object
REQUEST_TELE            40482 non-null object
REQUEST_RESP            40482 non-null object
REQUEST_CDIFF           40482 non-null object
REQUEST_MRSA            40482 non-null object
REQUEST_VRE             40482 non-null object
CALLOUT_OUTCOME         40482 non-null object
DBSOURCE                40482 non-null object
FIRST_CAREUNIT          40482 non-null object
LAST_CAREUNIT           40482 non-null object
GENDER                  40482 non-null object
admTime_weekday  